In [1]:
import pandas as pd
import joblib

# api활용을 위해 제공되고 있는 xycode에서 부산 code만 추출한다

In [2]:
data = pd.read_excel('../Data/rawdata/API_xycode.xlsx')

In [3]:
data['CODE'] = data['행정구역코드'].apply(str)

In [4]:
busan_data = data[data['CODE'].str.startswith('26')]

In [5]:
busan_data = busan_data[['2단계','3단계','격자 X', '격자 Y', 'CODE']]

In [6]:
busan_data = busan_data.fillna(0)
busan_data = busan_data[busan_data['3단계']==0]

In [7]:
busan_data = busan_data.drop(columns='3단계')
busan_data = busan_data.drop(index=451)
busan_data

2단계  격자 X  격자 Y        CODE
452    중구    97    74  2611000000
462    서구    97    74  2614000000
476    동구    98    75  2617000000
489   영도구    98    74  2620000000
501  부산진구    97    75  2623000000
522   동래구    98    76  2626000000
536    남구    98    75  2629000000
554    북구    96    76  2632000000
568  해운대구    99    75  2635000000
587   사하구    96    74  2638000000
604   금정구    98    77  2641000000
621   강서구    96    76  2644000000
630   연제구    98    76  2647000000
643   수영구    99    75  2650000000
654   사상구    96    75  2653000000
667   기장군   100    77  2671000000

In [8]:
busan_data = busan_data.rename(columns={'2단계':'ZONE','격자 X':'X','격자 Y' : 'Y'})

In [9]:
busan = busan_data.reset_index(drop=True)

In [10]:
busan

ZONE    X   Y        CODE
0     중구   97  74  2611000000
1     서구   97  74  2614000000
2     동구   98  75  2617000000
3    영도구   98  74  2620000000
4   부산진구   97  75  2623000000
5    동래구   98  76  2626000000
6     남구   98  75  2629000000
7     북구   96  76  2632000000
8   해운대구   99  75  2635000000
9    사하구   96  74  2638000000
10   금정구   98  77  2641000000
11   강서구   96  76  2644000000
12   연제구   98  76  2647000000
13   수영구   99  75  2650000000
14   사상구   96  75  2653000000
15   기장군  100  77  2671000000

In [11]:
busan_data.to_csv('../Data/final_data/DB/xycode.csv', encoding='utf-8', index=False)

# busan_dong을 위한 코드를 만든다.

In [12]:
busan_dong = pd.read_csv('../Data/processing_data/1st/busan_flood_geo.csv')

busan_dong = busan_dong[['ADM_CD','ZONE','DONG','SLOPE_AVG','HIGH','FLUD_DPWT.1']]

busan_dong = busan_dong.sort_values('FLUD_DPWT.1', ascending=False,ignore_index=True)

busan_dong = busan_dong.drop_duplicates(['DONG'], keep='first')\
.sort_values(['ZONE','DONG'], ascending=['FALSE','TRUE'])\
.reset_index(drop=True)

busan_dong = busan_dong.drop(columns='FLUD_DPWT.1')

new_data={'ADM_CD':2617053000,'ZONE':'동구','DONG':'초량동','SLOPE_AVG':'1','HIGH':'4'}
busan_dong = busan_dong.append(new_data, ignore_index=True)

busan_dong = busan_dong[['ADM_CD','ZONE','DONG','SLOPE_AVG','HIGH']]
busan_dong['SLOPE_AVG'] = busan_dong['SLOPE_AVG'].astype(float)
busan_dong['HIGH'] = busan_dong['HIGH'].astype(float)

In [13]:
slope_weight = busan_dong['SLOPE_AVG'].copy()
high_weight = busan_dong['HIGH'].copy()

#경사도가중치
for idx,col in enumerate(busan_dong['SLOPE_AVG']):
    if col <= 10:
        slope_weight[idx] = col / (41.67 / 5**2 )
    elif col > 10 and col <= 15:
        slope_weight[idx] = col / (25 / 4**2)
    elif col > 15 and col <= 25:
        slope_weight[idx] = col / (16.67 / 3**2)
    elif col > 25 and col <= 50:
        slope_weight[idx] = col / (8.33 / 2**2)
    elif col > 50:
        slope_weight[idx] = col / (8.33 / 1**2)
#고도 가중치
for idx,col in enumerate(busan_dong['HIGH']):
    if col <= 20:
        high_weight[idx] = col / (79.62 / 1**2)
    elif col > 20 and col <= 40:
        high_weight[idx] = col / (10.18 / 2**2)
    elif col > 40 and col <= 60:
        high_weight[idx] = col / (4.63 / 3**2)
    elif col > 60 and col <= 80:
        high_weight[idx] = col / (2.78 / 4**2)
    elif col > 80:
        high_weight[idx] = col / (2.78 / 5**2)
busan_dong['SLOPE_AVG_Weight'] = slope_weight
busan_dong['HIGH_Weight'] = high_weight
busan_dong = busan_dong.drop(columns=['SLOPE_AVG','HIGH'])

In [14]:
busan_dong

ADM_CD  ZONE   DONG  SLOPE_AVG_Weight  HIGH_Weight
0   2644012000   강서구    눌차동         12.147570     0.114418
1   2644000000   강서구   대저1동          0.599952     0.022231
2   2644000000   강서구   대저2동          0.599952     0.014820
3   2644010400   강서구    명지동          0.599952     0.019467
4   2644011900   강서구    성북동          7.040000     0.016453
5   2641000000   금정구    구서동          7.040000     9.803536
6   2641000000   금정구    금사동          2.699784     0.086410
7   2641000000   금정구    남산동          2.699784    14.231827
8   2641000000   금정구    노포동          7.040000   367.021583
9   2641000000   금정구    두구동          0.599952   356.661871
10  2641000000   금정구    부곡동          2.699784     0.162522
11  2641000000   금정구    장전동          2.699784     0.190404
12  2671000000   기장군    기장읍          2.699784     0.053253
13  2671000000   기장군    일광면          2.699784     0.025496
14  2671000000   기장군    장안읍          2.699784     0.061794
15  2671000000   기장군    정관읍          2.699784    13.972495
16  2671000000   기장군    철마면          2.699784   448.115108
17  2629000000    남구    용호동          0.599952     0.047350
18  2617000000    동구    범일동          0.599952     0.037177
19  2626000000   동래구    명륜동          0.599952     0.095956
20  2626000000   동래구    사직동          1.799856     0.082015
21  2626000000   동래구    수안동          0.599952     0.058905
22  2626000000   동래구    안락동          7.040000     0.064557
23  2626000000   동래구    온천동          0.599952     0.135770
24  2623000000  부산진구    가야동          2.699784    13.135560
25  2632000000    북구    구포동         21.608643  1538.129496
26  2632000000    북구    덕천동          7.040000     0.125220
27  2632000000    북구    화명동          9.600000    12.687623
28  2653000000   사상구    감전동          0.599952     0.030646
29  2653000000   사상구    괘법동          0.599952     0.034665
30  2653000000   사상구    덕포동          0.599952     0.072972
31  2653000000   사상구    주례동          0.599952     0.017081
32  2638010800   사하구    감천동          5.099592     0.080633
33  2638010100   사하구    괴정동          8.000000    10.416503
34  2638000000   사하구    구평동          0.599952     0.031650
35  2638000000   사하구    다대동          0.599952     0.036925
36  2638010500   사하구    장림동          0.599952     0.024868
37  2638010300   사하구    하단동          0.599952     0.030143
38  2614000000    서구    암남동          7.040000     0.024115
39  2650000000   수영구    광안동          2.699784     0.042452
40  2650010500   수영구    남천동          2.699784     0.039186
41  2650000000   수영구    망미동          7.040000     0.072846
42  2650000000   수영구    민락동          2.699784     0.025245
43  2650000000   수영구    수영동          0.599952     0.043582
44  2647000000   연제구    거제동          2.699784     0.072218
45  2647000000   연제구    연산동          0.599952     0.043080
46  2620010700   영도구  남항동3가          0.599952     0.024366
47  2620000000   영도구    동삼동         21.608643    12.298625
48  2620012000   영도구    청학동          0.599952     0.039940
49  2635000000  해운대구    반여동          2.699784     0.033534
50  2635000000  해운대구    석대동          0.599952     0.078875
51  2635000000  해운대구    송정동          2.699784    10.569745
52  2635000000  해운대구     우동          0.599952     0.043582
53  2635000000  해운대구     중동          2.699784     0.083773
54  2617053000    동구    초량동          0.599952     0.050239

In [15]:
sig_info = pd.read_csv('../Data/processing_data/2nd/sig_info_weight.csv')

In [16]:
busan_dong = pd.merge(busan_dong, sig_info, on='ZONE', how='left')

In [17]:
busan_dong.to_csv('../Data/final_data/DB/busan_base_data.csv', encoding='utf-8', index=False)

# REALdata 학습한 데이터 만들기

In [18]:
simulation_rain = pd.read_csv('../Data/rawdata/simulation_rain.csv')

In [19]:
simulation_rain.rename(columns={'DONG':'ZONE'}, inplace=True)

In [20]:
simulation_rain

ZONE  RAINFALL  HOUR
0    강서구       0.0     0
1    강서구       0.0     1
2    강서구       0.0     2
3    강서구       0.0     3
4    강서구       0.0     4
..   ...       ...   ...
415  연제구      20.5    23
416  연제구      37.0    24
417  연제구       0.5    25
418  연제구       0.0    26
419  연제구       0.0    27

[420 rows x 3 columns]

In [21]:

rain_weight = simulation_rain['RAINFALL'].copy()

## 강우량 가중치

for idx,col in enumerate(simulation_rain['RAINFALL']):
    if col <= 15:
        rain_weight[idx] = col / (46)
    elif col > 15 and col <= 35:
        rain_weight[idx] = col / (18 * 2**2)
    elif col > 35 and col <= 60:
        rain_weight[idx] = col / (17 * 3**2)
    elif col > 60 and col <= 80:
        rain_weight[idx] = col / (11 * 4**2)
    elif col > 80:
        rain_weight[idx] = col / (6.28 * 5**2)

In [22]:
simulation_rain['Hourly_Rainfall_Weight'] = rain_weight

In [23]:
simulation_data = pd.merge(busan_dong, simulation_rain,on='ZONE',how='left')

In [24]:
simulation_data

ADM_CD ZONE DONG  SLOPE_AVG_Weight  HIGH_Weight        CODE  \
0     2644012000  강서구  눌차동         12.147570     0.114418  2644000000   
1     2644012000  강서구  눌차동         12.147570     0.114418  2644000000   
2     2644012000  강서구  눌차동         12.147570     0.114418  2644000000   
3     2644012000  강서구  눌차동         12.147570     0.114418  2644000000   
4     2644012000  강서구  눌차동         12.147570     0.114418  2644000000   
...          ...  ...  ...               ...          ...         ...   
1535  2617053000   동구  초량동          0.599952     0.050239  2617000000   
1536  2617053000   동구  초량동          0.599952     0.050239  2617000000   
1537  2617053000   동구  초량동          0.599952     0.050239  2617000000   
1538  2617053000   동구  초량동          0.599952     0.050239  2617000000   
1539  2617053000   동구  초량동          0.599952     0.050239  2617000000   

        PUMP_RATIO  F_WEIGHT  Impervious_Surface_Weight  RAINFALL  HOUR  \
0     1.189961e-07  0.075019                   0.609000       0.0     0   
1     1.189961e-07  0.075019                   0.609000       0.0     1   
2     1.189961e-07  0.075019                   0.609000       0.0     2   
3     1.189961e-07  0.075019                   0.609000       0.0     3   
4     1.189961e-07  0.075019                   0.609000       0.0     4   
...            ...       ...                        ...       ...   ...   
1535  1.982161e-07  0.016667                   0.434346      13.3    23   
1536  1.982161e-07  0.016667                   0.434346       8.0    24   
1537  1.982161e-07  0.016667                   0.434346       0.1    25   
1538  1.982161e-07  0.016667                   0.434346       0.0    26   
1539  1.982161e-07  0.016667                   0.434346       0.0    27   

      Hourly_Rainfall_Weight  
0                   0.000000  
1                   0.000000  
2                   0.000000  
3                   0.000000  
4                   0.000000  
...                      ...  
1535                0.289130  
1536                0.173913  
1537                0.002174  
1538                0.000000  
1539                0.000000  

[1540 rows x 12 columns]

In [25]:
simulation_data.columns

Index(['ADM_CD', 'ZONE', 'DONG', 'SLOPE_AVG_Weight', 'HIGH_Weight', 'CODE',
       'PUMP_RATIO', 'F_WEIGHT', 'Impervious_Surface_Weight', 'RAINFALL',
       'HOUR', 'Hourly_Rainfall_Weight'],
      dtype='object')

In [26]:
simulation_data = simulation_data.sort_values('HOUR')
simulation_data.reset_index(drop=True, inplace=True)

In [27]:
sim_data = simulation_data[['PUMP_RATIO','Hourly_Rainfall_Weight','Impervious_Surface_Weight','SLOPE_AVG_Weight',\
                           'HIGH_Weight','F_WEIGHT']]

# 준비된 데이터로 모델 불러와서 예측

In [28]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA    

In [29]:
model = joblib.load('../Data/final_data/model/geo_model/ensemble.pkl')

In [30]:
scaler = MinMaxScaler()
scaler.fit(sim_data)
sim_data = scaler.transform(sim_data)

pca = PCA(n_components=4)
printcipalComponents = pca.fit_transform(sim_data)
sim_data = pd.DataFrame(data=printcipalComponents, columns = ['principal component1', 'principal component2','principal component3', 'principal component4'])

y_pred = model.predict_proba(sim_data)

In [31]:
y_pred

array([[0.96045664, 0.03954336],
       [0.73671261, 0.26328739],
       [0.59232543, 0.40767457],
       ...,
       [0.65504698, 0.34495302],
       [0.36019218, 0.63980782],
       [0.69991007, 0.30008993]])

In [32]:
item = list()

In [33]:
for k in y_pred:
    item.append(k[1])

In [34]:
k = pd.Series(item)

In [35]:
simulation_data = pd.concat([simulation_data,k],axis=1)

In [36]:
simulation_data

ADM_CD ZONE DONG  SLOPE_AVG_Weight  HIGH_Weight        CODE  \
0     2644012000  강서구  눌차동         12.147570     0.114418  2644000000   
1     2671000000  기장군  일광면          2.699784     0.025496  2671000000   
2     2650000000  수영구  민락동          2.699784     0.025245  2650000000   
3     2671000000  기장군  기장읍          2.699784     0.053253  2671000000   
4     2650000000  수영구  수영동          0.599952     0.043582  2650000000   
...          ...  ...  ...               ...          ...         ...   
1535  2638000000  사하구  구평동          0.599952     0.031650  2638000000   
1536  2641000000  금정구  구서동          7.040000     9.803536  2641000000   
1537  2653000000  사상구  감전동          0.599952     0.030646  2653000000   
1538  2641000000  금정구  금사동          2.699784     0.086410  2641000000   
1539  2617053000   동구  초량동          0.599952     0.050239  2617000000   

        PUMP_RATIO  F_WEIGHT  Impervious_Surface_Weight  RAINFALL  HOUR  \
0     1.189961e-07  0.075019                   0.609000       0.0     0   
1     1.364381e-08  0.016667                   1.156156       0.0     0   
2     0.000000e+00  0.075019                   0.460742       0.5     0   
3     1.364381e-08  0.016667                   1.156156       0.0     0   
4     0.000000e+00  0.075019                   0.460742       0.5     0   
...            ...       ...                        ...       ...   ...   
1535  1.485884e-07  0.149925                   0.955239       0.0    27   
1536  3.042751e-08  0.016667                   0.473500       0.0    27   
1537  3.862069e-07  0.016667                   0.301755       0.0    27   
1538  3.042751e-08  0.016667                   0.473500       0.0    27   
1539  1.982161e-07  0.016667                   0.434346       0.0    27   

      Hourly_Rainfall_Weight         0  
0                    0.00000  0.039543  
1                    0.00000  0.263287  
2                    0.01087  0.407675  
3                    0.00000  0.263287  
4                    0.01087  0.518198  
...                      ...       ...  
1535                 0.00000  0.199586  
1536                 0.00000  0.054617  
1537                 0.00000  0.344953  
1538                 0.00000  0.639808  
1539                 0.00000  0.300090  

[1540 rows x 13 columns]

In [37]:
simulation_data['predict'] = simulation_data[0]
simulation_result = simulation_data.pivot_table(index='DONG',columns='HOUR',values='predict')

In [38]:
simulation_result.to_csv('../Data/final_data/DB/Simulation_Result.csv',encoding='utf-8')